Instructions:
-

1. Read the article: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
2. Replicate the study using the same dataset.
3. Read articles about Adjusted Rand Index, Normalized Mutual Information, and Folkes-Mallows Index (only use paper published in IEEE, sciencedirect, springerlink, Taylor Francis).
4. Aside from the Adjusted Rand Index (ARI), and Normalized Mutual Information (NMI), use the Folkes-Mallows Index (FMI), and compare the result of each performance index.
5. Compare and contrast each performance index, what are the advantages and disadvantages of ARI, NMI, and FMI, and when to use each?
6. Using Kmodes and Hierarchical Clustering, use the same dataset and perform categorical data clustering, use FMI, ARI, and NMI for the comparison of performance.
7. Write your report using Latex. Your report should be focused on the "why's and the what's" of each performance metrices (i.e. why is FMI always greater than ARI and NMI? What's the problem with ARI and NMI?).

In [ ]:
# Import necessary libraries
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import random
import requests
from itertools import combinations
from io import StringIO
from kmodes.kmodes import KModes
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.impute import SimpleImputer
from sklearn.manifold import SpectralEmbedding, TSNE
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI, fowlkes_mallows_score as FMI
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import layers, Sequential
import warnings

# Adjust the Pandas display options for better readability
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 300)

# Ignore all warnings
warnings.filterwarnings('ignore')

# List of datasets and their corresponding URLs
dataset_urls = {
    "Soybean (Small)": "https://archive.ics.uci.edu/ml/machine-learning-databases/soybean/soybean-small.data",
    "Zoo": "https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data",
    "Heart Disease": "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
    "Breast Cancer Wisconsin (Original)": "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",
    "Dermatology": "https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data",
    "Letter Recognition (E, F)": "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data",
    "Molecular Biology (Splice-junction Gene Sequences)": "https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/splice-junction-gene-sequences/splice.data",
    "Mushroom": "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
    "Iris": "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
    "ISOLET (5)": "https://mikaelvincent.dev/datasets/isolet/isolet5.data",
    "Optical Recognition of Handwritten Digits": "https://mikaelvincent.dev/datasets/opticaldigits/optdigits.data",
    "Pen-Based Recognition of Handwritten Digits": "https://mikaelvincent.dev/datasets/pendigits/pendigits.data"
}
# COIL20 not implemented; I don't understand how images can be translated into categories; I can't find a pre-processed version either :)

# Set the true number of clusters for each dataset
n_clusters_dict = {
    "Soybean (Small)": 4,
    "Zoo": 7,
    "Heart Disease": 2,
    "Breast Cancer Wisconsin (Original)": 2,
    "Dermatology": 6,
    "Letter Recognition (E, F)": 2,
    "Molecular Biology (Splice-junction Gene Sequences)": 3,
    "Mushroom": 2,
    "Iris": 3,
    "ISOLET (5)": 26,
    "Optical Recognition of Handwritten Digits": 10,
    "Pen-Based Recognition of Handwritten Digits": 10
}

# List of datasets for clustering ensemble task
datasets_for_ensemble = [
    "Iris",
    "ISOLET (5)",
    "Optical Recognition of Handwritten Digits",
    "Pen-Based Recognition of Handwritten Digits"
]

# Set the number of runs for benchmarking
num_runs = 10

n_runs = 60  # Number of k-means runs for clustering ensemble

dataframes = {}  # Dictionary to store cleaned dataframes

for name, url in dataset_urls.items():
    response = requests.get(url, verify=False)
    data = response.text
    
    # Convert the CSV/Text data into a DataFrame
    data_io = StringIO(data)
    df = pd.read_csv(data_io, header=None)

    # Set targets and features
    if name == "Letter Recognition (E, F)":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Mushroom":
        y = df.iloc[:, 0]
        X = df.iloc[:, 1:]
    elif name == "Molecular Biology (Splice-junction Gene Sequences)":
        y = df.iloc[:, 0].str.strip()
        X = pd.DataFrame([list(seq.strip()) for seq in df.iloc[:, 2]])
    else:
        X, y = df.iloc[:, :-1], df.iloc[:, -1]

    # Drop columns with only 1 unique value
    for col in X.columns:
        if len(X[col].unique()) <= 1:
            X.drop(columns=[col], inplace=True) # Diregard warning as it is behaving as expected
    
    # Store in the dataframes dictionary
    dataframes[name] = {'features': X, 'targets': y}

def preprocess_datasets(dataframes):
    if 'Zoo' in dataframes:
        zoo_df = dataframes['Zoo']['features']
        zoo_df = zoo_df.drop(columns=[0])
        dataframes['Zoo']['features'] = zoo_df

    if 'Heart Disease' in dataframes:
        hd_df = dataframes['Heart Disease']['features']
        columns_to_drop = [0, 3, 4, 7, 9]
        hd_df = hd_df.drop(columns=hd_df.columns[columns_to_drop])
        dataframes['Heart Disease']['features'] = hd_df
        y_hd = dataframes['Heart Disease']['targets']
        dataframes['Heart Disease']['targets'] = y_hd.apply(lambda x: 0 if x == 0 else 1)
    
    if 'Breast Cancer Wisconsin (Original)' in dataframes:
        bcw_df = dataframes['Breast Cancer Wisconsin (Original)']['features']
        bcw_df = bcw_df.drop(columns=bcw_df.columns[0])
        dataframes['Breast Cancer Wisconsin (Original)']['features'] = bcw_df
    
    if 'Dermatology' in dataframes:
        derm_df = dataframes['Dermatology']['features']
        derm_df = derm_df.drop(columns=derm_df.columns[-1])
        dataframes['Dermatology']['features'] = derm_df

    if 'Letter Recognition (E, F)' in dataframes:
        lr_ef_df = dataframes['Letter Recognition (E, F)']['features']
        lr_ef_targets = dataframes['Letter Recognition (E, F)']['targets']
        mask = lr_ef_targets.isin(['E', 'F'])
        dataframes['Letter Recognition (E, F)']['features'] = lr_ef_df[mask]
        dataframes['Letter Recognition (E, F)']['targets'] = lr_ef_targets[mask]

    return dataframes

# Apply preprocessing to datasets
dataframes = preprocess_datasets(dataframes)

def run_multiple_kmeans(features, n_clusters, n_runs):
    all_labels = []
    for i in range(n_runs):
        random_state = random.randint(0, 1000)
        kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=random_state, n_init=10)
        labels = kmeans.fit_predict(features)
        all_labels.append(labels)
    return np.array(all_labels).T

for dataset_name in datasets_for_ensemble:
    # Extracting features and targets from the preloaded datasets
    features = dataframes[dataset_name]["features"]
    targets = dataframes[dataset_name]["targets"]

    # Converting targets to numerical labels if they aren't already
    if targets.dtype.kind in 'O':  # Check if targets are object type (e.g., strings)
        targets = LabelEncoder().fit_transform(targets)

    # Determine the number of clusters from the unique elements in targets
    n_clusters = len(np.unique(targets))

    # Run multiple k-means and collect results
    ensembled_features = run_multiple_kmeans(features, n_clusters, n_runs)
    
    # Convert numpy array to DataFrame and replace the original data
    ensembled_features_df = pd.DataFrame(ensembled_features, index=features.index)
    dataframes[dataset_name]["features"] = ensembled_features_df

def perform_kmodes(features, n_clusters):
    """Perform clustering using KModes algorithm."""
    km = KModes(n_clusters=n_clusters, init='random', n_init=5)
    clusters = km.fit_predict(features)
    return clusters

def perform_ordinal_encoding(features, true_labels, n_clusters):
    """Perform Ordinal Encoding followed by clustering."""
    encoder = LabelEncoder()
    features_encoded = features.apply(encoder.fit_transform)
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    return kmeans.fit_predict(features_encoded, n_clusters)

def perform_one_hot_encoding(features, true_labels, n_clusters):
    """Perform One-Hot Encoding followed by clustering."""
    encoder = OneHotEncoder()
    features_encoded = encoder.fit_transform(features).toarray()
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    return kmeans.fit_predict(features_encoded, n_clusters)

def perform_link(features, n_clusters):
    encoder = OneHotEncoder()
    features_encoded = encoder.fit_transform(features).toarray()
    # Calculate pairwise dissimilarities (1 - similarity), ensuring non-negative distances
    ochiai_distance = pdist(features_encoded, lambda u, v: max(0, 1 - ochiai_coefficient_for_link(u, v)))
    link_matrix = linkage(ochiai_distance, method='average')
    clusters = fcluster(link_matrix, t=n_clusters, criterion='maxclust')
    return clusters

def ochiai_coefficient_for_link(b1, b2):
    intersection = np.dot(b1, b2)
    norm_b1 = np.sqrt(np.dot(b1, b1))
    norm_b2 = np.sqrt(np.dot(b2, b2))
    denominator = (norm_b1 * norm_b2)
    if denominator == 0:
        return 0  # Return 0 if either or both vectors are all zeros
    return intersection / denominator

def perform_cde(features, n_clusters):
    """Perform Categorical Data Embedding and clustering using t-SNE and k-Means."""
    encoder = OneHotEncoder(sparse_output=False)
    features_encoded = encoder.fit_transform(features)
    tsne_model = TSNE(n_components=2, perplexity=30, learning_rate=200)
    features_embedded = tsne_model.fit_transform(features_encoded)
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    clusters = kmeans.fit_predict(features_embedded)
    return clusters

def perform_hierarchical_clustering(features, n_clusters, method='ward'):
    """Perform Hierarchical Clustering with handling for '?' characters and categorical variables."""
    
    # Handle '?' characters and replace them appropriately
    for column in features.columns:
        if features[column].dtype == object:
            # Check if the column contains '?'
            if '?' in features[column].unique():
                if features[column].str.isnumeric().any():
                    # Assume numeric column with missing values represented as '?'
                    # Convert '?' to NaN and then fill with the mean of the column
                    features[column] = pd.to_numeric(features[column], errors='coerce')
                    features[column].fillna(features[column].mean(), inplace=True)
                else:
                    # Categorical column
                    mode_value = features[column].mode()[0]
                    features[column] = features[column].replace('?', mode_value)
            # Encode categorical variables
            le = LabelEncoder()
            features[column] = le.fit_transform(features[column].astype(str))
        else:
            # Directly fill missing values with mean if any
            if features[column].isnull().any():
                features[column].fillna(features[column].mean(), inplace=True)

    # Ensure all data is in float format to avoid conversion errors in linkage
    features = features.astype(float)

    # Create the linkage matrix
    Z = linkage(features, method=method)

    # Create clusters by cutting the dendrogram at the specified number of clusters
    clusters = fcluster(Z, t=n_clusters, criterion='maxclust')
    return clusters

def perform_cdc_dr(features, n_clusters, embedding_method='SE', operation='Joint'):
    """
    Perform CDC_DR algorithm with specified graph embedding method and operation.
    :param features: DataFrame of features
    :param n_clusters: Number of clusters to form
    :param embedding_method: 'NE', 'SE', 'NMF', or 'AE'
    :param operation: 'Joint' or 'Mean'
    :return: clusters - Cluster labels for each sample
    """
    
    # Construct similarity graph from features
    graph = construct_similarity_graph(features)

    # Apply graph embedding technique
    embedded_graph = graph_embedding(graph, method=embedding_method)

    value_to_index = create_value_to_index_mapping(features)
    
    # Ensure integrated_data is 2D before clustering
    if operation == 'Joint':
        integrated_data = joint_operation(embedded_graph, features, value_to_index)
    elif operation == 'Mean':
        integrated_data = mean_operation(embedded_graph, features, value_to_index)
    else:
        raise ValueError("Operation must be either 'Joint' or 'Mean'.")
    
    # Impute missing values in integrated_data
    imputer = SimpleImputer(strategy='mean')
    integrated_data_imputed = imputer.fit_transform(integrated_data)
    
    # Cluster the integrated data
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    clusters = kmeans.fit_predict(integrated_data_imputed)

    return clusters

def construct_similarity_graph(features):
    """
    Construct a similarity graph from features based on categorical values.
    :param features: DataFrame of features, each row is a sample and columns are categorical features
    :return: graph - A NetworkX graph with nodes representing categorical values and weighted edges
    """
    # Step 1: Prepare all unique categorical values and their indices
    unique_values_dict = {}
    for column in features:
        unique_values = np.unique(features[column])
        for val in unique_values:
            unique_values_dict[f"{column}_{val}"] = np.where(features[column] == val)[0]
    
    # Step 2: Calculate similarity between all pairs of unique categorical values
    graph = nx.Graph()
    for (val1, indices1), (val2, indices2) in combinations(unique_values_dict.items(), 2):
        # Calculate similarity (e.g., using Ochiai coefficient)
        sim = ochiai_coefficient(indices1, indices2)  # Define this function based on your chosen similarity metric
        if sim > 0:  # If the similarity is non-zero, add an edge
            graph.add_edge(val1, val2, weight=sim)
    
    # Add all nodes explicitly in case some have no edges
    for val in unique_values_dict.keys():
        if val not in graph:
            graph.add_node(val)

    return graph

def ochiai_coefficient(indices1, indices2):
    """
    Calculate Ochiai coefficient between two sets of indices
    :param indices1: array-like list of indices for the first categorical value
    :param indices2: array-like list of indices for the second categorical value
    :return: Ochiai coefficient as float
    """
    set1 = set(indices1)
    set2 = set(indices2)
    intersection = len(set1.intersection(set2))
    if intersection == 0: return 0  # No overlap
    return intersection / np.sqrt(len(set1) * len(set2))  # Ochiai coefficient formula

def graph_embedding(graph, method='SE', dimensions=2):
    """
    Apply graph embedding method to the constructed graph.
    :param graph: NetworkX graph
    :param method: string representing the graph embedding method: 'NE', 'SE', 'NMF', 'AE'
    :param dimensions: the number of dimensions for the embedding
    :return: embedded_graph - An array-like embedded representation of the graph
    """
    # Convert graph to adjacency matrix and then to numpy ndarray
    adjacency_matrix = nx.to_numpy_matrix(graph)
    adjacency_matrix = np.asarray(adjacency_matrix)
    
    if method == 'NE':
        # Directly use the adjacency matrix as features (no embedding)
        embedded_graph = adjacency_matrix

    elif method == 'SE':
        # Apply Spectral Embedding
        embedding_model = SpectralEmbedding(n_components=dimensions)
        embedded_graph = embedding_model.fit_transform(adjacency_matrix)

    elif method == 'NMF':
        # Apply Non-negative Matrix Factorization for embedding
        model = NMF(n_components=dimensions, init='random', max_iter=10000)
        embedded_graph = model.fit_transform(adjacency_matrix)

    elif method == 'AE':
        # Apply Autoencoder for graph embedding
        n_nodes = adjacency_matrix.shape[0]
        input_dim = n_nodes
        autoencoder = Sequential([
            layers.Dense(64, activation='relu', input_shape=(input_dim,)),
            layers.Dense(dimensions, activation='relu'),  # Embedding layer
            layers.Dense(64, activation='relu'),
            layers.Dense(input_dim, activation='sigmoid')
        ])
        autoencoder.compile(optimizer='adam', loss='mse')
        adjacency_matrix_norm = adjacency_matrix / np.max(adjacency_matrix)  # Normalize adjacency matrix
        autoencoder.fit(adjacency_matrix_norm, adjacency_matrix_norm, epochs=50, verbose=0)
        encoder = Sequential(autoencoder.layers[:2])  # The first two layers are the encoder
        embedded_graph = encoder.predict(adjacency_matrix_norm, verbose=0)

    else:
        raise NotImplementedError(f"Graph embedding method {method} is not implemented.")
    
    return np.array(embedded_graph)

def create_value_to_index_mapping(features):
    """
    Create a mapping from each unique categorical value to a unique index.
    :param features: DataFrame of features, each column is a categorical feature
    :return: Dictionary of value to index mapping
    """
    # Extracting unique values from each feature
    unique_values = set()
    for column in features.columns:
        unique_values.update(features[column].unique())

    # Creating a mapping from unique values to an index
    value_to_index = {value: idx for idx, value in enumerate(unique_values)}
    return value_to_index

def joint_operation(embedded_graph, features, value_to_index):
    # Concatenates the embeddings for each categorical value in each sample
    joint_embedded = []
    for _, row in features.iterrows():
        joint_vector = []
        for value in row:
            index = value_to_index[value]  # Map each categorical value to its index in the embedded graph
            joint_vector.extend(embedded_graph[index])
        joint_embedded.append(joint_vector)
    return np.array(joint_embedded)

def mean_operation(embedded_graph, features, value_to_index):
    # Calculates the mean of the embeddings for each categorical value in each sample
    mean_embedded = []
    for _, row in features.iterrows():
        vectors = [embedded_graph[value_to_index[value]] for value in row]
        mean_vector = np.mean(vectors, axis=0)
        mean_embedded.append(mean_vector)
    return np.array(mean_embedded)


def run_clustering_algorithms(dataframes, n_clusters_dict, num_runs=10):
    results_list = []
    for name, data in dataframes.items():
        print("Processing:", name)
        features = data['features']
        true_labels = data['targets'].squeeze()  # Assuming targets are in a single column
        n_clusters = n_clusters_dict.get(name, 2)  # Default to 2 clusters if not specified

        metrics = {'KModes': [], 'Ordinal': [], 'One-Hot': [], 'Link': [], 'CDE': [], 'Hierarchical': []}  # Initialize a dictionary to store results for each method

        # Include CDC_DR methods in metrics dictionary
        embedding_methods = ['NE', 'SE', 'NMF', 'AE']  # Non-Embedding, Spectral Embedding, Nonnegative Matrix Factorization, Autoencoder
        operations = ['Joint', 'Mean']  # The two types of operations
        
        for em in embedding_methods:
            for op in operations:
                key_name = f"CDC_DR+{em} ({op})"
                metrics[key_name] = []

        for _ in range(num_runs):
            # KModes
            km_clusters = perform_kmodes(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, km_clusters)
            metrics['KModes'].append((ari, nmi, fmi))

            # Ordinal Encoding
            ord_clusters = perform_ordinal_encoding(features, true_labels, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, ord_clusters)
            metrics['Ordinal'].append((ari, nmi, fmi))

            # One-Hot Encoding
            oh_clusters = perform_one_hot_encoding(features, true_labels, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, oh_clusters)
            metrics['One-Hot'].append((ari, nmi, fmi))

            # Link with Ochiai Coefficient
            link_clusters = perform_link(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, link_clusters)
            metrics['Link'].append((ari, nmi, fmi))

            # CDE with t-SNE and k-Means
            cde_clusters = perform_cde(features, n_clusters)
            ari, nmi, fmi = calculate_metrics(true_labels, cde_clusters)
            metrics['CDE'].append((ari, nmi, fmi))

            # Hierarchical Clustering
            hier_clusters = perform_hierarchical_clustering(features, n_clusters)
            ari, nmi, fmi = ARI(true_labels, hier_clusters), NMI(true_labels, hier_clusters), FMI(true_labels, hier_clusters)
            metrics['Hierarchical'].append((ari, nmi, fmi))

            # CDC_DR with various embedding methods and operations
            for embedding_method in ['NE', 'SE', 'NMF', 'AE']:
                for operation in ['Joint', 'Mean']:
                    cdc_dr_clusters = perform_cdc_dr(features, n_clusters, embedding_method, operation)
                    ari, nmi, fmi = calculate_metrics(true_labels, cdc_dr_clusters)
                    metrics[f"CDC_DR+{embedding_method} ({operation})"].append((ari, nmi, fmi))

        # Calculate mean and standard deviation for each method and append to results list
        for method, values in metrics.items():
            ari_vals, nmi_vals, fmi_vals = zip(*values)
            ari_mean, ari_std = np.mean(ari_vals), np.std(ari_vals)
            nmi_mean, nmi_std = np.mean(nmi_vals), np.std(nmi_vals)
            fmi_mean, fmi_std = np.mean(fmi_vals), np.std(fmi_vals)
            results_list.append({
                "Dataset": name,
                "Method": method,
                "ARI": f"{ari_mean:.4f}±{ari_std:.2f}",
                "NMI": f"{nmi_mean:.4f}±{nmi_std:.2f}",
                "FMI": f"{fmi_mean:.4f}±{fmi_std:.2f}"
            })

    # Convert list of dictionaries to DataFrame for results
    results_df = pd.DataFrame(results_list)
    return results_df

def calculate_metrics(true_labels, predicted_labels):
    """Calculate clustering metrics: Adjusted Rand Index, Normalized Mutual Information, and Folkes-Mallows Index."""
    return ARI(true_labels, predicted_labels), NMI(true_labels, predicted_labels), FMI(true_labels, predicted_labels)

def reformat_results(results_df):
    # Expanding 'ARI', 'NMI', and 'FMI' columns into multiple rows with a new 'Metric' column
    expanded_df = pd.melt(results_df, id_vars=["Dataset", "Method"], value_vars=["ARI", "NMI", "FMI"], var_name="Metric", value_name="Value")
    expanded_df[['Metric_Value', 'Std']] = expanded_df['Value'].str.split('±', expand=True)
    expanded_df.drop(columns=['Value'], inplace=True)  # Removing the original combined column
    
    # Convert the 'Metric_Value' and 'Std' columns to numeric types
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].astype(float)
    expanded_df['Std'] = expanded_df['Std'].astype(float)

    # Concatenate the metric value and standard deviation back into a single string
    expanded_df['Metric_Value'] = expanded_df['Metric_Value'].map('{:.4f}'.format) + "±" + expanded_df['Std'].map('{:.2f}'.format)
    
    # Ensuring the order of datasets and methods remains consistent with the original DataFrame
    dataset_order = results_df['Dataset'].unique()
    method_order = results_df['Method'].unique()

    # Creating a pivot table to restructure the DataFrame as required
    pivot_df = expanded_df.pivot_table(index=["Dataset", "Metric"], columns="Method", values="Metric_Value", aggfunc='first')
    
    # Reindexing the pivot table to maintain the original order
    pivot_df = pivot_df.reindex(dataset_order, level='Dataset')
    pivot_df = pivot_df.reindex(method_order, axis='columns')

    return pivot_df

# Running all algorithms and storing the results
results = run_clustering_algorithms(dataframes, n_clusters_dict, num_runs)

Processing: Soybean (Small)
WARNING:tensorflow:5 out of the last 9 calls to <function Model.make_predict_function.<locals>.predict_function at 0x000001723BAC0790> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING:tensorflow:6 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x000001723BAC0CA0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Processing: Zoo
Processing: Heart Disease
Processing: Breast Cancer Wisconsin (Original)
Processing: Dermatology
Processing: Letter Recognition (E, F)
Processing: Molecular Biology (Splice-junction Gene Sequences)
Processing: Mushroom
Processing: Iris
Processing: ISOLET (5)
Processing: Optical Recognition of Handwritten Digits
Processing: Pen-Based Recognition of Handwritten Digits

# Use the function to reformat the results
formatted_results = reformat_results(results)

Section: Presentation of Results

# Prepare data for the table
table_data = []

for name, content in dataframes.items():
    X, y = content['features'], content['targets']
    table_data.append({
        "Name": name,
        "Number of Samples": X.shape[0],
        "Number of Features": X.shape[1],
        "Number of Unique Values in Target": len(pd.unique(y))
    })

# Convert table data into a DataFrame for pretty printing
table_df = pd.DataFrame(table_data)
print(table_df)

print("\n* Number of features for numerical datasets are based on the # of runs of kmeans in the clustering ensemble. For an accurate measure of the original versions of the datasets, run this before running the clustering ensemble.")

                                                 Name  Number of Samples  Number of Features  Number of Unique Values in Target
0                                     Soybean (Small)                 47                  21                                  4
1                                                 Zoo                101                  16                                  7
2                                       Heart Disease                303                   8                                  2
3                  Breast Cancer Wisconsin (Original)                699                   9                                  2
4                                         Dermatology                366                  33                                  6
5                           Letter Recognition (E, F)               1543                  16                                  2
6   Molecular Biology (Splice-junction Gene Sequen...               3190                  60                                  3
7                                            Mushroom               8124                  21                                  2
8                                                Iris                150                  60                                  3
9                                          ISOLET (5)               1559                  60                                 26
10          Optical Recognition of Handwritten Digits               5620                  60                                 10
11        Pen-Based Recognition of Handwritten Digits              10992                  60                                 10

* Number of features for numerical datasets are based on the # of runs of kmeans in the clustering ensemble. For an accurate measure of the original versions of the datasets, run this before running the clustering ensemble.

# Print the formatted results
print(formatted_results)

Method                                                          KModes      Ordinal      One-Hot          Link          CDE Hierarchical CDC_DR+NE (Joint) CDC_DR+NE (Mean) CDC_DR+SE (Joint) CDC_DR+SE (Mean) CDC_DR+NMF (Joint) CDC_DR+NMF (Mean) CDC_DR+AE (Joint) CDC_DR+AE (Mean)
Dataset                                            Metric                                                                                                                                                                                                                             
Soybean (Small)                                    ARI     0.9097±0.14  0.5458±0.00  1.0000±0.00   1.0000±0.00  0.9420±0.08  1.0000±0.00       1.0000±0.00      0.0120±0.01       0.5562±0.00      0.1079±0.00        1.0000±0.00      -0.0046±0.00       0.7776±0.32      0.0013±0.03
                                                   FMI     0.9319±0.11  0.6573±0.00  1.0000±0.00   1.0000±0.00  0.9565±0.06  1.0000±0.00       1.0000±0.00      0.2768±0.00       0.6692±0.00      0.3795±0.00        1.0000±0.00       0.2788±0.00       0.8598±0.18      0.3101±0.10
                                                   NMI     0.9422±0.09  0.7160±0.00  1.0000±0.00   1.0000±0.00  0.9540±0.06  1.0000±0.00       1.0000±0.00      0.1369±0.01       0.7248±0.00      0.2554±0.00        1.0000±0.00       0.0843±0.00       0.8351±0.29      0.0746±0.05
Zoo                                                ARI     0.6810±0.09  0.7315±0.12  0.7156±0.09   0.8893±0.00  0.6483±0.04  0.6958±0.00       0.7455±0.09      0.3029±0.04       0.8175±0.07      0.1677±0.01        0.7086±0.07       0.1502±0.01       0.5810±0.20      0.1682±0.11
                                                   FMI     0.7519±0.07  0.7916±0.09  0.7813±0.07   0.9162±0.00  0.7291±0.03  0.7641±0.00       0.8041±0.07      0.4422±0.03       0.8597±0.05      0.3353±0.01        0.7741±0.06       0.3242±0.00       0.7011±0.09      0.3924±0.07
                                                   NMI     0.7955±0.03  0.8023±0.04  0.8410±0.03   0.8731±0.00  0.8262±0.02  0.7935±0.00       0.8393±0.03      0.4744±0.04       0.8665±0.02      0.3751±0.01        0.8113±0.03       0.3607±0.00       0.7222±0.24      0.3213±0.17
Heart Disease                                      ARI     0.3733±0.02  0.4515±0.01  0.4234±0.01   0.2915±0.10  0.3513±0.03  0.3707±0.00       0.4381±0.00      0.2661±0.00       0.1412±0.00      0.0146±0.00        0.2661±0.00       0.1402±0.03       0.2567±0.15      0.1072±0.07
                                                   FMI     0.6924±0.01  0.7284±0.01  0.7119±0.00   0.6680±0.01  0.6766±0.01  0.6856±0.00       0.7238±0.00      0.6331±0.00       0.5715±0.00      0.5407±0.00        0.6335±0.00       0.5707±0.01       0.6737±0.04      0.5779±0.05
                                                   NMI     0.2942±0.02  0.3566±0.01  0.3346±0.00   0.2227±0.07  0.2721±0.02  0.2889±0.00       0.3482±0.00      0.2049±0.00       0.1117±0.00      0.0250±0.00        0.2025±0.00       0.1048±0.02       0.1998±0.12      0.0882±0.05
Breast Cancer Wisconsin (Original)                 ARI     0.7104±0.05  0.8173±0.00  0.7983±0.00   0.0026±0.00  0.7898±0.06  0.8502±0.00       0.7579±0.00      0.7087±0.00       0.6118±0.00      0.6065±0.00        0.7172±0.00       0.6650±0.01       0.6547±0.02      0.2849±0.14
                                                   FMI     0.8752±0.02  0.9183±0.00  0.9064±0.00   0.7397±0.00  0.9027±0.03  0.9314±0.00       0.8871±0.00      0.8638±0.00       0.8167±0.00      0.8150±0.00        0.8685±0.00       0.8441±0.00       0.8374±0.01      0.6617±0.07
                                                   NMI     0.6064±0.04  0.7100±0.00  0.7249±0.00   0.0047±0.00  0.7021±0.06  0.7606±0.00       0.6883±0.00      0.6297±0.00       0.5542±0.00      0.5239±0.00        0.6200±0.00       0.5606±0.01       0.5956±0.02      0.2293±0.13
Dermatology                                        ARI     0.5092±0.06  0.7321±0.06  0.8516±0.11   0.5468±0.00  0.7136±0.02  0.7121±0.00       0.6873±0.05      0.1862±0.00       0.6269±0.06      0.1528±0.00        0.7086±0.06       0.1799±0.00       0.6703±0.05      0.1403±0.07
                                                   FMI     0.6069±0.04  0.7852±0.05  0.8811±0.09   0.6971±0.00  0.7686±0.02  0.7691±0.00       0.7480±0.05      0.3408±0.00       0.6990±0.05      0.3121±0.00        0.7662±0.05       0.3370±0.00       0.7354±0.04      0.3541±0.05
                                                   NMI     0.6136±0.05  0.8805±0.01  0.9074±0.03   0.7161±0.00  0.8100±0.01  0.8763±0.00       0.7850±0.04      0.3180±0.00       0.6514±0.04      0.2761±0.01        0.7952±0.03       0.3109±0.00       0.7574±0.05      0.2413±0.12
Letter Recognition (E, F)                          ARI     0.1992±0.07  0.5200±0.00  0.2564±0.01   0.2108±0.00  0.6563±0.01  0.0500±0.00       0.6006±0.01      0.2130±0.00       0.2000±0.00      0.1594±0.00        0.1997±0.00       0.1141±0.01       0.2055±0.07      0.2072±0.07
                                                   FMI     0.6097±0.04  0.7647±0.00  0.6296±0.01   0.6594±0.00  0.8280±0.00  0.5356±0.00       0.8037±0.00      0.6064±0.00       0.6006±0.00      0.5801±0.00        0.6004±0.00       0.5569±0.00       0.6040±0.03      0.6255±0.03
                                                   NMI     0.1663±0.06  0.4632±0.00  0.1975±0.01   0.3098±0.00  0.5478±0.01  0.0397±0.00       0.5509±0.00      0.1603±0.00       0.1512±0.00      0.1194±0.00        0.1509±0.00       0.0844±0.00       0.1565±0.05      0.1587±0.05
Molecular Biology (Splice-junction Gene Sequences) ARI     0.0266±0.01  0.0418±0.03  0.5781±0.03  -0.0074±0.00  0.1913±0.04  0.0637±0.00       0.6148±0.02      0.0576±0.00       0.3813±0.00      0.0281±0.00        0.2772±0.01       0.0527±0.00       0.3393±0.07      0.0450±0.01
                                                   FMI     0.3770±0.01  0.4127±0.00  0.7335±0.02   0.6108±0.00  0.4820±0.03  0.4213±0.00       0.7564±0.01      0.3958±0.00       0.6050±0.00      0.3943±0.00        0.5411±0.00       0.3928±0.00       0.5775±0.05      0.3987±0.00
                                                   NMI     0.0414±0.02  0.0626±0.03  0.5319±0.02   0.0119±0.00  0.2321±0.05  0.0552±0.00       0.5669±0.02      0.0680±0.00       0.3693±0.00      0.0475±0.00        0.2481±0.01       0.0608±0.00       0.3425±0.07      0.0584±0.01
Mushroom                                           ARI     0.6138±0.00  0.1729±0.00  0.6186±0.00   0.0009±0.00  0.3642±0.12  0.1695±0.00       0.6147±0.00      0.0324±0.00       0.3205±0.00      0.0179±0.00        0.4783±0.11       0.0407±0.00       0.2523±0.08      0.0216±0.01
                                                   FMI     0.8123±0.00  0.6409±0.00  0.8143±0.00   0.7040±0.00  0.6855±0.06  0.6377±0.00       0.8123±0.00      0.5175±0.00       0.6604±0.00      0.5124±0.00        0.7440±0.05       0.5214±0.00       0.6654±0.02      0.5116±0.00
                                                   NMI     0.5697±0.01  0.1935±0.00  0.5692±0.00   0.0109±0.00  0.3056±0.10  0.1854±0.00       0.5620±0.00      0.0232±0.00       0.2459±0.00      0.0141±0.00        0.4102±0.11       0.0292±0.00       0.2565±0.05      0.0155±0.00
Iris                                               ARI     0.7302±0.00  0.7302±0.00  0.7302±0.00   0.7302±0.00  0.7302±0.00  0.7302±0.00       0.7302±0.00      0.7302±0.00       0.7302±0.00      0.7302±0.00        0.3651±0.37       0.1460±0.29       0.0000±0.00      0.3651±0.37
                                                   FMI     0.8208±0.00  0.8208±0.00  0.8208±0.00   0.8208±0.00  0.8208±0.00  0.8208±0.00       0.8208±0.00      0.8208±0.00       0.8208±0.00      0.8208±0.00        0.6971±0.12       0.6229±0.10       0.5735±0.00      0.6971±0.12
                                                   NMI     0.7582±0.00  0.7582±0.00  0.7582±0.00   0.7582±0.00  0.7582±0.00  0.7582±0.00       0.7582±0.00      0.7582±0.00       0.7582±0.00      0.7582±0.00        0.3791±0.38       0.1516±0.30       0.0000±0.00      0.3791±0.38
ISOLET (5)                                         ARI     0.4385±0.02  0.4635±0.01  0.4669±0.00   0.4737±0.00  0.4631±0.01  0.4534±0.00       0.4728±0.01      0.3353±0.01       0.4349±0.01      0.2076±0.01        0.4614±0.01       0.2092±0.01       0.4534±0.02      0.2210±0.02
                                                   FMI     0.4646±0.02  0.4859±0.01  0.4890±0.00   0.4983±0.00  0.4849±0.01  0.4767±0.00       0.4945±0.01      0.3632±0.01       0.4585±0.01      0.2438±0.01        0.4835±0.01       0.2440±0.01       0.4761±0.02      0.2546±0.02
                                                   NMI     0.7053±0.01  0.7224±0.00  0.7258±0.00   0.7270±0.00  0.7237±0.01  0.7213±0.00       0.7258±0.00      0.5974±0.01       0.7053±0.01      0.4663±0.00        0.7217±0.00       0.4651±0.00       0.7166±0.01      0.4836±0.02
Optical Recognition of Handwritten Digits          ARI     0.6704±0.00  0.6704±0.00  0.6704±0.00   0.6704±0.00  0.6837±0.00  0.6707±0.00       0.6704±0.00      0.6685±0.00       0.6737±0.01      0.6152±0.03        0.6706±0.00       0.5188±0.04       0.6705±0.00      0.6272±0.04
                                                   FMI     0.7063±0.00  0.7063±0.00  0.7063±0.00   0.7063±0.00  0.7164±0.00  0.7066±0.00       0.7063±0.00      0.7046±0.00       0.7090±0.01      0.6594±0.03        0.7065±0.00       0.5860±0.03       0.7064±0.00      0.6696±0.03
                                                   NMI     0.7566±0.00  0.7562±0.00  0.7566±0.00   0.7566±0.00  0.7503±0.00  0.7570±0.00       0.7566±0.00      0.7523±0.00       0.7569±0.00      0.7079±0.02        0.7565±0.00       0.6582±0.02       0.7565±0.00      0.7208±0.02
Pen-Based Recognition of Handwritten Digits        ARI     0.5820±0.03  0.5648±0.00  0.5319±0.00   0.5319±0.00  0.5533±0.00  0.5638±0.00       0.5320±0.00      0.4933±0.00       0.5541±0.03      0.4751±0.03        0.5612±0.01       0.4969±0.02       0.5498±0.03      0.4819±0.02
                                                   FMI     0.6273±0.02  0.6108±0.00  0.5874±0.00   0.5874±0.00  0.6005±0.00  0.6098±0.00       0.5874±0.00      0.5569±0.00       0.6056±0.03      0.5415±0.02        0.6086±0.01       0.5561±0.02       0.6016±0.02      0.5457±0.02
                                                   NMI     0.6914±0.01  0.6794±0.00  0.6820±0.00   0.6820±0.00  0.6678±0.00  0.6788±0.00       0.6821±0.00      0.6432±0.00       0.6892±0.01      0.6175±0.02        0.6831±0.00       0.6210±0.02       0.6869±0.01      0.6185±0.02

Questions

The Adjusted Rand Index (ARI), Normalized Mutual Information (NMI), and Fowlkes-Mallows Index (FMI) are all metrics used to evaluate the performance of clustering algorithms by comparing the clustering results with ground truth labels.
Adjusted Rand Index (ARI)

    Advantages: Corrects for chance grouping, making it more reliable in indicating actual similarities between clusters and ground truth. Values range from -1 to 1, where 1 indicates perfect agreement, and values less than 0 indicate independent labelings.
    Disadvantages: Can be more computationally intensive than simpler metrics; less intuitive to interpret without statistical background.
    Use Case: Best for benchmarking different clustering algorithms when a ground truth is available.

Normalized Mutual Information (NMI)

    Advantages: Also adjusts for chance and is normalized between 0 and 1, where 1 indicates a perfect match between clusters and ground truth. It measures mutual information in a way that accounts for the sizes of different clusters.
    Disadvantages: Its normalization can sometimes lead to misleading results, especially if the number of clusters is very different from the number of ground truth categories.
    Use Case: Useful in cases where clusters are of varying sizes and a normalized measure is preferred.

Fowlkes-Mallows Index (FMI)

    Advantages: Based on the geometric mean of precision and recall, providing a balance between them. Simple to calculate and understand, with values ranging from 0 to 1, where 1 indicates perfect clustering.
    Disadvantages: Does not adjust for chance, which can make it overly optimistic in scenarios where random clustering might appear effective.
    Use Case: Effective for quick assessments where computational simplicity is needed and the number of clusters is close to the number of categories in ground truth.

When to Use Each

    ARI is preferred when an accurate and chance-corrected measure is critical.
    NMI is suitable for comparing clusters of varying sizes and when normalization of mutual information is beneficial.
    FMI is good for quick, intuitive assessments of clustering when the expected number of clusters is not significantly different from the number of ground truth categories.

